For this homework, make sure that you format your notbook nicely and cite all sources in the appropriate sections. Programmatically generate or embed any figures or graphs that you need.

Names: __Zhen Guo and Vikram Chowdhary__

## DO THIS

Step 1: Train your own word embeddings
--------------------------------

We chose to use the provided Spooky Author dataset. It contains text from works of fiction written by "spooky authors" of the public domain - Edgar Allan poe, HP Lovecraft, and Mary Shelley. The features in this dataset are:
- id - a unique identifier for each sentence
- text - some text written by one of the authors
- author - the author of the sentence (EAP: Edgar Allan Poe, HPL: HP Lovecraft; MWS: Mary Wollstonecraft Shelley)
The training portion of this dataset has 19579 texts, and the testing portion has 8392

Describe what data set you have chosen to compare and contrast with the your chosen provided dataset. Make sure to describe where it comes from and it's general properties.

The dataset we selected was found on Kaggle, and consists of 50,000 IMDB reviews. https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

In [17]:
# import your libraries here
import pandas as pd
import nltk
import re
from nltk.stem import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
# !pip install gensim

### 0) Pre-processing and text-normalization

The following pre-processing steps are inspired from https://towardsdatascience.com/text-normalization-for-natural-language-processing-nlp-70a314bfa646.

We also pre-processed data so that it begins with < s> tokens (and ends with < /s> tokens). Inspired from answer: https://stackoverflow.com/questions/37605710/tokenize-a-paragraph-into-sentence-and-then-into-words-in-nltk

In [14]:
# normalize text to regular expression
# code from https://gist.github.com/yamanahlawat/4443c6e9e65e74829dbb6b47dd81764a

replacement_patterns = [
  (r'won\'t', 'will not'),
  (r'can\'t', 'cannot'),
  (r'i\'m', 'i am'),
  (r'ain\'t', 'is not'),
  (r'(\w+)\'ll', '\g<1> will'),
  (r'(\w+)n\'t', '\g<1> not'),
  (r'(\w+)\'ve', '\g<1> have'),
  (r'(\w+)\'s', '\g<1> is'),
  (r'(\w+)\'re', '\g<1> are'),
  (r'(\w+)\'d', '\g<1> would')
]

patterns = [(re.compile(regex), repl) for (regex, repl) in replacement_patterns]

def replace(text):
    s = text
    for (pattern, repl) in patterns:
        s = re.sub(pattern, repl, s)
    return s

In [31]:
def process_text(text):
    """
    Process the paragram so it is tokenized into sentences, 
    each sentence start with <s> end withh </s>, words are tokenized and normalized for each sentence.
    """
    sent_text = nltk.sent_tokenize(text) # this gives us a list of sentences
    
    # now loop over each sentence and tokenize it separately
    s = []
    for sentence in sent_text:
        # regualr expression
        sentence = replace(sentence)
        # tokenize sentence
        tokenized_text = nltk.word_tokenize(sentence)

        # lematizing and stemming words
        ps = SnowballStemmer("english")
        lemmatizer = WordNetLemmatizer()
        
        new_sent = ['<s>']
        for word in tokenized_text:
            # now remove punctuation
            if not word.isalpha():
                continue           
            # stemming:
            word = ps.stem(word)
            # lemmatizing
            word = lemmatizer.lemmatize(word)
            new_sent.append(word)

        # add begin and end
        new_sent.append('</s>')
        s = s + new_sent
    return s

def process_data(series):
    # returns text in this format:
    # data = [['this', 'is', 'the', 'first', 'sentence', 'for', 'word2vec'],
    # 			['this', 'is', 'the', 'second', 'sentence'],
    # 			['yet', 'another', 'sentence'],
    # 			['one', 'more', 'sentence'],
    # 			['and', 'the', 'final', 'sentence']]
    sentences = []
    for _,row in series.items():
        sentences.append(process_text(row))
    
    return sentences

In [32]:
# Read the file and prepare the training data 
# so that it is in the following format
spooky_authors_train = pd.read_csv('./spooky-author-identification/train.csv')
spooky_authors_test = pd.read_csv('./spooky-author-identification/test.csv')
df_imdb = pd.read_csv('IMDB_dataset.csv')

given_data_train = process_data(spooky_authors_train['text'])
given_data_test = process_data(spooky_authors_test['text'])
our_data = process_data(df_imdb['review'])

In [40]:
# save the variables in case leter use
# should be comment out before submission
import pickle

with open('our_data.pickle', 'wb') as f:
    pickle.dump(our_data, f)

### a) Train embeddings on GIVEN dataset

In [34]:
from gensim.models import Word2Vec

# The dimension of word embedding. 
# This variable will be used throughout the program
# you may vary this as you desire
EMBEDDINGS_SIZE = 200

# Train the Word2Vec model from Gensim. 
# Below are the hyperparameters that are most relevant. 
# But feel free to explore other 
# options too:
# sg = 1
# window = 5
# size = EMBEDDINGS_SIZE
# min_count = 1
# train model on spooky authors training data
model = Word2Vec(sentences = given_data_train, 
                 vector_size = EMBEDDINGS_SIZE,
                 sg = 1, 
                 window = 5,  
                 min_count = 1)

In [35]:
# if you save your Word2Vec as the variable model, this will 
# print out the vocabulary size
# THIS DOES NOT WORK?
# print('Vocab size {}'.format(len(model.wv.vocab)))
# https://stackoverflow.com/questions/35596031/gensim-word2vec-find-number-of-words-in-vocabulary
print('Vocab size {}'.format(len(model.wv)))

Vocab size 14996


In [36]:
# You can save file in txt format, then load later if you wish.
model.wv.save_word2vec_format('embeddings.txt', binary=False)

### b) Train embedding on YOUR dataset

In [37]:
# then do a second data set
# given data is roughly 70/30 train/test
our_data_train = our_data[:35000]
our_data_test = our_data[35000:]
our_model = Word2Vec(sentences = our_data_train, 
                     vector_size = EMBEDDINGS_SIZE,
                     sg = 1, 
                     window = 5,  
                     min_count = 1)

In [38]:
print('Vocab size {}'.format(len(our_model.wv)))

Vocab size 56862


In [39]:
# You can save file in txt format, then load later if you wish.
our_model.wv.save_word2vec_format('imdb_embeddings.txt', binary=False)

What text-normalization and pre-processing did you do and why? __Sentence begin and end, regular expression expansion, tokenization, punctuation removing, lemmatizing and stemming__

Step 2: Evaluate the differences between the word embeddings
----------------------------

(make sure to include graphs, figures, and paragraphs with full sentences)

In [ ]:
# model.wv.index_to_key
# get 10 most common and uncommon words/vectors? plot difference between them?

In [ ]:
model.wv.word_index

AttributeError: 'KeyedVectors' object has no attribute 'word_index'

##Write down your analysis:

Cite your sources:
-------------

Step 3: Feedforward Neural Language Model
--------------------------

# a) First, encode  your text into integers

In [41]:
# Importing utility functions from Keras
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import SimpleRNN
from keras.layers import Embedding

# The size of the ngram language model you want to train
# change as needed for your experiments
N_GRAM = 3 

# Initializing a Tokenizer
# It is used to vectorize a text corpus. Here, it just creates a mapping from 
# word to a unique index. (Note: Indexing starts from 0)
# Example:
# tokenizer = Tokenizer()
# tokenizer.fit_on_texts(data)
# encoded = tokenizer.texts_to_sequences(data)


2022-11-22 23:52:39.782682: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [52]:
spooky_tokenizer = Tokenizer()
imdb_tokenizer = Tokenizer()

# spooky authors data
# use out own tokenizer
spooky_train_list = given_data_train
spooky_tokenizer.fit_on_texts(spooky_train_list)
spooky_train_encoded = spooky_tokenizer.texts_to_sequences(spooky_train_list)

# # our data
# imdb_train_list = list(df_imdb['review'].values)[:35000]
# imdb_tokenizer.fit_on_texts(imdb_train_list)
# imdb_train_encoded = tokenizer.texts_to_sequences(imdb_train_list)

### b) Next, prepare your sequences from text

#### Fixed ngram based sequences 

In [55]:
def generate_ngram_training_samples(ngram_len: int, data: list) -> list:
    '''
    Takes the encoded data (list of lists) and 
    generates the training samples out of it.
    Parameters:
    up to you, we've put in what we used
    but you can add/remove as needed
    return: 
    list of lists in the format [[x1, x2, ... , x(n-1), y], ...]
    '''
    # TODO: does this make sense????
    final_ngrams = []
    for idx in range(len(data) - ngram_len):
        ngram_list = list(data[idx:idx+ngram_len])
        final_ngrams.append(ngram_list)
    return final_ngrams


### c) Then, split the sequences into X and y and create a Data Generator

In [56]:
# Note here that the sequences were in the form: 
# sequence = [x1, x2, ... , x(n-1), y]
# We still need to separate it into [[x1, x2, ... , x(n-1)], ...], [y1, y2, ...]
def split_ngrams(ngram_list: list) -> list:
    x = []
    y = []
    for ngram in ngram_list:
        y.append(ngram[-1])
        x.append(ngram[:-1])
    return x, y

In [72]:
import string

def read_embeddings(text, embeddings, tokenizer):
    '''Loads and parses embeddings trained in earlier.
    Parameters and return values are up to you.
    '''
    
    # you may find generating the following two dicts useful:
    # word to embedding : {'the':[0....], ...}
    # index to embedding : {1:[0....], ...} 
    # use your tokenizer's word_index to find the index of
    # a given word
    word_to_embedding = dict()
    index_to_embedding = dict()
    tok_w_i = tokenizer.word_index
    for sentence in text:
        for word in sentence:
            # since we already pre processed data, we no longer need to transform
            if word not in word_to_embedding.keys():
                #print(word)
                word_to_embedding[word] = embeddings.wv[word]
                index_to_embedding[tok_w_i[word]] = embeddings.wv[word]
                
    return word_to_embedding, index_to_embedding

In [73]:
a, b = read_embeddings(spooky_train_list, model, spooky_tokenizer)

In [75]:
# embedding of word process
a['process']

array([ 0.02348194,  0.05490553, -0.00265442,  0.07038224,  0.10827535,
       -0.09777361,  0.05405985,  0.2373621 ,  0.02425046,  0.00219891,
       -0.07679407, -0.18995433,  0.04362183, -0.03777902, -0.07255875,
       -0.05782289, -0.0801895 ,  0.12139725, -0.10765211, -0.29708388,
        0.08327043,  0.04365621, -0.01183267,  0.03152594, -0.0836893 ,
       -0.01145581, -0.00815305, -0.04344771, -0.05997968, -0.01018504,
        0.12941797, -0.01724505,  0.13043666, -0.04741822, -0.07351422,
        0.03786735,  0.19337331,  0.06287841,  0.03567098, -0.15253997,
       -0.07178811, -0.01642449, -0.01931473,  0.06069664,  0.14860804,
       -0.0097814 ,  0.00895152, -0.03115348,  0.06453577,  0.08881211,
        0.08606131, -0.15818632, -0.06499314, -0.19137977, -0.02956738,
       -0.04876706,  0.05038584, -0.12518013, -0.06588586,  0.04635868,
       -0.00166237, -0.07724451,  0.11163273,  0.05546798, -0.22543083,
        0.1496334 , -0.10520876,  0.09416441, -0.19911195,  0.19

In [76]:
# embedding of token of index 4
b[4]

array([ 0.15042208,  0.0618179 ,  0.0828281 ,  0.20855531,  0.4031695 ,
       -0.09448706,  0.0366256 ,  0.59077597,  0.1094589 ,  0.06333733,
        0.2052773 , -0.00314236, -0.04288219,  0.01718473,  0.02472387,
       -0.1329551 ,  0.04697161,  0.13526043, -0.05162509, -0.37635106,
       -0.01000359, -0.05844728,  0.20229833, -0.05589949, -0.11722791,
       -0.26393858, -0.04103449, -0.17500141, -0.08418668, -0.2092509 ,
        0.09575804, -0.03705767,  0.2290568 , -0.05987259, -0.00166913,
       -0.01177609, -0.12572664, -0.13694699, -0.04620064, -0.22947   ,
       -0.02528268,  0.03157232,  0.00162438, -0.12686384,  0.03314819,
       -0.10010096, -0.06235256, -0.10038215,  0.11948317,  0.14735518,
        0.07228272, -0.02961588,  0.08497987, -0.03054579, -0.10082976,
       -0.14986514, -0.0655518 , -0.08715058,  0.07088859,  0.21680567,
       -0.2986772 , -0.13292263,  0.3267133 , -0.03025175, -0.31965357,
        0.16980793, -0.07496505,  0.191069  , -0.5702984 ,  0.15

In [ ]:
def data_generator(X: list, y: list, num_sequences_per_batch: int) -> (list,list):
    '''
    Returns data generator to be used by feed_forward
    https://wiki.python.org/moin/Generators
    https://realpython.com/introduction-to-python-generators/
    
    Yields batches of embeddings and labels to go with them.
    Use one hot vectors to encode the labels 
    (see the to_categorical function)
    
    '''
    pass



In [ ]:
# Examples
# initialize data_generator
# num_sequences_per_batch = 128 # this is the batch size
# steps_per_epoch = len(sequences)//num_sequences_per_batch  # Number of batches per epoch
# train_generator = data_generator(X, y, num_sequences_per_batch)

# sample=next(train_generator) # this is how you get data out of generators
# sample[0].shape # (batch_size, (n-1)*EMBEDDING_SIZE)  (128, 200)
# sample[1].shape   # (batch_size, |V|) to_categorical

### d) Train your models

In [ ]:
# code to train a feedforward neural language model 
# on a set of given word embeddings
# make sure not to just copy + paste to train your two models

# Define the model architecture using Keras Sequential API



In [ ]:
# Start training the model
model.fit(x=train_generator, 
          steps_per_epoch=steps_per_epoch,
          epochs=1)

AttributeError: 'Word2Vec' object has no attribute 'fit'

### e) Generate Sentences

In [ ]:
# generate a sequence from the model
def generate_seq(model: Sequential, 
                 tokenizer: Tokenizer, 
                 seed: list, 
                 n_words: int):
    '''
    Parameters:
        model: your neural network
        tokenizer: the keras preprocessing tokenizer
        seed: [w1, w2, w(n-1)]
        n_words: generate a sentence of length n_words
    Returns: string sentence
    '''
    pass

### f) Compare your generated sentences

Sources Cited
----------------------------
